In [1]:
import pandas as pd
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
# from util import UnitConversion, mapping_list_values, perl_to_posix
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
pd.options.display.max_rows = 500
from flashtext import KeywordProcessor
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
import ast
import warnings
warnings.filterwarnings('ignore')
from natsort import natsorted
def query_from_file(file_name, params):
    with open(f'{file_name}', mode='r') as f:
        text = f.read()
        query = text.format(**params)
        return enrich_db(query)
def re_extract(pattern, txt):
    matches = re.findall(pattern, txt)
    tmp_matches = []
    for match in matches:
        for tup in match:
            if tup != '':
                tmp_matches.append(tup)
    return list(set(tmp_matches))

# from enrich_dimensions.rounds import rounds, rounding, re_extract, curate, round_string_float, clean_data,reg_ex     
# from enrich_dimensions.params import parameters, query_from_file
# from enrich_dimensions.query_file import query_from_file 
# from enrich_dimensions.custom import custom_field 

# BBBY All Attributes

In [6]:
customer_id = '5'
customer_name='%bedbathandbeyond%'
dateszs='2022-06-20'

params = {'customer_id': customer_id,
          'customer_name':customer_name,
         'dateszs':dateszs}
bbby = query_from_file(file_name='../query/curated_all_attributes_all_customers_brackets.sql', params=params)
print(len(bbby))

94398


In [7]:
import time
today = time.strftime("%Y_%m_%d")
print(today)

wbm=bbby
trip=r'''(?i)(\{)|(\})|()'''                                                              
wbm['match']=wbm['value'].apply(lambda x: re_extract(trip,str(x)))
bracket=wbm[wbm['match'].astype(str)!='[]']#['value'].explode().value_counts()[0:500]
bracket['matches']=bracket['value'].apply(lambda x: re.sub(r'\{','[',str(x))).apply(lambda x: re.sub(r'\}',']',str(x)))
bracket['external_id'].explode().value_counts()
bracket

2022_06_21


,value,customer_name,external_id,Curated Date,Updated Date,match,matches


In [4]:
customer_id = '5'
customer_name='%bedbathandbeyond%'
dateszs='2001-02-05'

params = {'customer_id': customer_id,
          'customer_name':customer_name,
         'dateszs':dateszs}
bbby = query_from_file(file_name='../query/curated_all_attributes_date.sql', params=params)
print(len(bbby))

12502889


In [3]:
wbm=bbby
trip=r'''(?i)(\{)|(\})|()'''                                                              
wbm['match']=wbm['value'].apply(lambda x: re_extract(trip,str(x)))
bracket=wbm[wbm['match'].astype(str)!='[]']#['value'].explode().value_counts()[0:500]
bracket['matches']=bracket['value'].apply(lambda x: re.sub(r'\{','[',str(x))).apply(lambda x: re.sub(r'\}',']',str(x)))
bracket['attribute'].explode().value_counts()

height    4
depth     2
Name: attribute, dtype: int64

In [5]:
width=bracket[bracket['attribute'].astype(str)=='width']
depth=bracket[bracket['attribute'].astype(str)=='depth']
diameter=bracket[bracket['attribute'].astype(str)=='diameter']
height=bracket[bracket['attribute'].astype(str)=='height']
length=bracket[bracket['attribute'].astype(str)=='length']
cookware_size=bracket[bracket['attribute'].astype(str)=='cookware_size']

height['Q:height']=height['matches']
width['Q:width']=width['matches']
depth['Q:depth']=depth['matches']
diameter['Q:diameter']=diameter['matches']
length['Q:length']=length['matches']
cookware_size['Q:cookware_size']=cookware_size['matches']

match_height=height[['external_id','Q:height']]
match_width=width[['external_id','Q:width']]
match_depth=depth[['external_id','Q:depth']]
match_diameter=diameter[['external_id','Q:diameter']]
match_length=length[['external_id','Q:length']]
match_cookware_size=cookware_size[['external_id','Q:cookware_size']]

In [ ]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
def looks_good(customer, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/BBBY --{get_df_name(matches)}_bracket.csv',index=False) 
looks_good('Bed Bath & Beyond', match_height) 

In [12]:
car_seat=df[df['attribute'].astype(str)=='infant_car_seat_brand_model_compatibility']
trip=r'''(?i)(Graco)|(Nuna.?turtle)|(Chicco)|(UPPAbaby)|()'''                                                              
car_seat['match']=car_seat['product_name'].apply(lambda x: re_extract(trip,str(x)))
car_seat['matches']=car_seat['long_desc'].apply(lambda x: re_extract(trip,str(x)))
x=car_seat[(car_seat['match'].astype(str)!='[]')|(car_seat['matches'].astype(str)!='[]')]
print(len(x))

262


# Size

In [22]:
size=df[df['attribute'].astype(str)=='size']
print(len(size))

70259


In [33]:
size['buckets'].explode().value_counts().reset_index().rename(columns={'index':'Buckets','buckets':'Number of SKUs'}).to_csv('size_bucket_numbers.csv')

In [ ]:
size[size['value'].astype(str)=='0 to 12 Months']['buckets'].explode().value_counts()

size[size['value'].astype(str)=='0 to 24 Months']['buckets'].explode().value_counts()

size[size['value'].astype(str)=='0 to 3 Years']['buckets'].explode().value_counts()

size[size['value'].astype(str)=='0 to 4 Years']['buckets'].explode().value_counts()

size[size['value'].astype(str)=='12 to 36 Months']['buckets'].explode().value_counts()

size[size['value'].astype(str)=='0 to 24 Months']['buckets'].explode().value_counts()

In [153]:
import ast
def remove_duplicates(A):
    [A.pop(count) for count,elem in enumerate(A) if A.count(elem)!=1]
    return A

size['clean']=size['value'].apply(lambda x: re.sub(r'"\\"','',str(x))).apply(lambda x: re.sub(r'(?<=)\s?\-\s?(?=\d)',' to ',str(x)))
size['cleans']=size['clean'].apply(lambda x: f'["{x}"]')
size['cleans']=size['cleans'].apply(lambda x: re.sub(r'"?\]"\]','"]',str(x))).apply(lambda x: re.sub(r'\["\["?','["',str(x))).apply(lambda x: re.sub(r'"\,(?!")','","',str(x)))
size['match']=size['cleans'].apply(lambda x: re.sub(r'(?<!\d)(?:0 to )?3 Months','0 to 3 Months',str(x))).apply(lambda x: re.sub(r'(?<!to )(?<!\d)6 Months','3 to 6 Months',str(x))).apply(lambda x: re.sub(r'0 to 4 Months','["0 to 3 Months","3 to 6 Months"]',str(x))).apply(lambda x: re.sub(r'(?<!to )(18 Months?)','12 to 18 Months',str(x)))
size['match']=size['match'].apply(lambda x: re.sub(r'(?<!to )(?:9|10|11|12) Months','["9 to 12 Months"]',str(x))).apply(lambda x: re.sub(r'(0 to [789] Months)','["0 to 3 Months","3 to 6 Months","6 to 9 Months"]',str(x))).apply(lambda x: re.sub(r'(?<!\d)(?<!to )[456] Months','["3 to 6 Months"]',str(x))).apply(lambda x: re.sub(r'0 to 6 Months','["0 to 3 Months","3 to 6 Months"]',str(x)))
size['match']=size['match'].apply(lambda x: re.sub(r'0 to 12 Months','["0 to 3 Months","3 to 6 Months","6 to 9 Months","9 to 12 Months"]',str(x))).apply(lambda x: re.sub(r'12 to 24 months','["12 to 18 Months","18 to 24 Months"]',str(x)))
size['match']=size['match'].apply(lambda x: re.sub(r'(?<!to )24 months','18 to 24 Months',str(x))).apply(lambda x: re.sub(r'6 to 12 Months','["6 to 9 Months","9 to 12 Months"]',str(x))).apply(lambda x: re.sub(r'6 to 24 Months','["6 to 9 Months","9 to 12 Months","12 to 18 Months","18 to 24 Months"]',str(x)))
size['match']=size['match'].apply(lambda x: re.sub(r'12 to 36 months','["12 to 18 Months","18 to 24 Months","2 to 3 Years"]',str(x)))

size['match']=size['match'].apply(lambda x: re.sub(r'(?<!\d)1(?:Y|T)','1 Years',str(x))).apply(lambda x: re.sub(r'(?<!\d)2(?:Y|T)','2 Years',str(x))).apply(lambda x: re.sub(r'(?<!\d)3(?:Y|T)','3 Years',str(x))).apply(lambda x: re.sub(r'(?<!\d)4(?:Y|T)','4 Years',str(x))).apply(lambda x: re.sub(r'(?<!\d)5(?:Y|T)','5 Years',str(x)))
size['match']=size['match'].apply(lambda x: re.sub(r'(?<!\d)6(?:Y|T)','6 Years',str(x))).apply(lambda x: re.sub(r'(?<!\d)7(?:Y|T)','7 Years',str(x))).apply(lambda x: re.sub(r'(?<!\d)8(?:Y|T)','8 Years',str(x))).apply(lambda x: re.sub(r'(?<!\d)9(?:Y|T)','9 Years',str(x))).apply(lambda x: re.sub(r'(?<!\d)10(?:Y|T)','10 Years',str(x))).apply(lambda x: re.sub(r'(?<!\d)11(?:Y|T)','11 Years',str(x)))


size['match']=size['match'].apply(lambda x: f'["{x}"]').apply(lambda x: re.sub(r'\["\["','["',str(x))).apply(lambda x: re.sub(r'"\]"\]','"]',str(x))).apply(lambda x: re.sub(r'(?:"\])?"\,"(?:\[")?','","',str(x))).apply(lambda x: re.sub(r'"\]"\]','"]',str(x))).apply(lambda x: re.sub(r'\["\["','["',str(x)))
size['cleaned']=size['match'].apply(lambda x: remove_duplicates([n.strip() for n in ast.literal_eval(x)])).apply(lambda x: natsorted(x))

In [ ]:
# size_index=size['cleaned'].explode().value_counts().reset_index().sort_values('index',ascending=True)
# pattern=r'''(?i)(to)|(month(?!s And))|(toddler)|(Preemie)|(new.?born)|(infant)|(year)|(\dT)|(\dY)|()'''                                                              
# size_index['match']=size_index['index'].apply(lambda x: re_extract(pattern,str(x)))
# sized=size_index[size_index['match'].astype(str)!='[]']
# del sized['match']
# sized

In [29]:
# trip=r'''(?i)(\d+T)|()'''                                                              
# size['t']=size['value'].apply(lambda x: re_extract(trip,str(x)))
# print('SKUs with T: '+str(len(size[size['t'].astype(str)!='[]'])))
# print('Buckets with T: '+str(size[size['t'].astype(str)!='[]']['buckets'].explode().value_counts().reset_index()['index'].to_list()))

# print('')
# print('######################################')
# print('')

# trip=r'''(?i)(\d+Y)|()'''                                                              
# size['y']=size['value'].apply(lambda x: re_extract(trip,str(x)))
# print('SKUs with Y: '+str(len(size[size['y'].astype(str)!='[]'])))
# print('Buckets with Y: '+str(size[size['y'].astype(str)!='[]']['buckets'].explode().value_counts().reset_index()['index'].to_list()))

In [28]:
# size['t'].explode().value_counts()

In [27]:
# size['y'].explode().value_counts()

In [26]:
# size['buckets'].explode().value_counts().reset_index()['index'].to_list()

In [25]:
# size[size['buckets'].astype(str)=='Baby Bibs & Burp Cloths']['value'].explode().value_counts()
# size[(size['buckets'].astype(str)=='Protective/Active Bibs')&((size['value'].astype(str)=='Small')|(size['value'].astype(str)=='X-Small')|(size['value'].astype(str)=='X-Large')|(size['value'].astype(str)=='Large')|(size['value'].astype(str)=='Medium'))]

In [ ]:
trip=r'''(\d+\-\d+(?:T|Y)(?![A-z]))|(\d+M\-\d+M)|(\d+\s?M?\-\d+\s?M(?:onths))|(\d+(?:\s?to\s?|\s?\-\s?)\d+\s?(?:m(?:onths?)?|years?))|((?:1[0-8]|(?<!\d)[1-9])(?:\.5)?(?:W|Y))|(\d+\s?Months?(?: and up)?)|([1-9]T)|(1[0-9]T)|(adult)|(infant)|((?:X.)?large)|(medium)|(newborn)|(one.?size)|(preemie)|(small)|(toddler)|(youth)|((?!<\w)X{1,4}(?:L|S))|(Size.?(?:[1-9]|1[0-9])M)|(Carhartt.?.?Shortall)|(extra.?small)|(\dX.?L)|()'''                                                              
size['match']=size['product_name'].apply(lambda x: re_extract(trip,str(x)))
print(len(size[size['match'].astype(str)!='[]']))
uncurated=size[size['match'].astype(str)!='[]']
uncurated=uncurated[uncurated['value'].astype(str)=='n/a']

In [ ]:
uncurated['clean']=uncurated['match'].apply(lambda x: f'["{x}"]')
uncurated['clean']=uncurated['clean'].apply(lambda x: re.sub(r'"?\]"\]','"]',str(x))).apply(lambda x: re.sub(r'\["\["?','["',str(x))).apply(lambda x: re.sub(r'"\,(?!")','","',str(x)))

uncurated['matchesss']=uncurated['clean'].apply(lambda x: re.sub(r'(12M-18M)','12 Months to 18 Months',str(x))).apply(lambda x: re.sub(r'\["2T?\-4T"\]','["2T","3T","4T"]',str(x))).apply(lambda x: re.sub(r'\["2\-3Y"\]','["2T","3T"]',str(x))).apply(lambda x: re.sub(r'(\["3\-4T"\])','["3T","4T"]',str(x))).apply(lambda x: re.sub(r'2XL','XX-Large',str(x))).apply(lambda x: re.sub(r'3XL','XXX-Large',str(x))).apply(lambda x: re.sub(r'4XL','XXXX-Large',str(x))).apply(lambda x: re.sub(r'','',str(x))).apply(lambda x: re.sub(r'','',str(x))).apply(lambda x: re.sub(r'','',str(x))).apply(lambda x: re.sub(r'','',str(x))).apply(lambda x: re.sub(r'','',str(x))).apply(lambda x: re.sub(r'','',str(x))).apply(lambda x: re.sub(r'','',str(x)))

uncurated['matchesss']=uncurated['matchesss'].apply(lambda x: re.sub(r'2\-4T','["2T","3T","4T"]',str(x))).apply(lambda x: re.sub(r'(?<![A-z])\s?(?<![A-z])\-\s?(?!\dT)(?!\dY)',' to ',str(x))).apply(lambda x: re.sub(r'2\-4Y','["2Y","3Y","4Y"]',str(x))).apply(lambda x: re.sub(r'newborn','Newborn',str(x))).apply(lambda x: re.sub(r'large','Large',str(x))).apply(lambda x: re.sub(r'medium','Medium',str(x))).apply(lambda x: re.sub(r'(?i)((?<=\d)M)|(Month(?!s))',' Months',str(x))).apply(lambda x: re.sub(r'(?i)(size\s?)|(\d+\-)','',str(x))).apply(lambda x: re.sub(r"(\s?\'\s?)",'"',str(x))).apply(lambda x: re.sub(r'Carhartt\® Shortall','',str(x))).apply(lambda x: re.sub(r'(?<=)Mns',' Months',str(x))).apply(lambda x: re.sub(r'\["\["','["',str(x))).apply(lambda x: re.sub(r'"\]"\]','"]',str(x))).apply(lambda x: re.sub(r'"?(?:\])?","(?:\[)"?','',str(x)))                                                                       

uncurated['matchesss']=uncurated['matchesss'].apply(lambda x: re.sub(r'""','"',str(x))).apply(lambda x: re.sub(r'"?\]"\]','"]',str(x))).apply(lambda x: re.sub(r'\["\["?','["',str(x))).apply(lambda x: re.sub(r'"\,(?!")','","',str(x)))

import ast
def remove_duplicates(A):
    [A.pop(count) for count,elem in enumerate(A) if A.count(elem)!=1]
    return A
uncurated['matchez']=uncurated['matchesss'].apply(lambda x: remove_duplicates([n.strip() for n in ast.literal_eval(x)])).apply(lambda x: natsorted(x))
uncurated['matchez']=uncurated['matchez'].apply(lambda x: re.sub(r"XL",'X-Large',str(x))).apply(lambda x: re.sub(r"\s?'\s?",'"',str(x))).apply(lambda x: re.sub(r'\[""\]','',str(x))).apply(lambda x: re.sub(r'one','One Size',str(x))).apply(lambda x: re.sub(r'  ',' ',str(x)))
uncurated['matchez']=uncurated['matchez'].apply(lambda x: re.sub(r'18 Months","18 Months','18 Months',str(x))).apply(lambda x: re.sub(r'6 Months","6 to 9 Months','6 to 9 Months',str(x))).apply(lambda x: re.sub(r'(9 to 12 Months","12 Months)|(9 Months","9 to 12 Months)','9 to 12 Months',str(x))).apply(lambda x: re.sub(r'9 to 12 Months","12 Months','9 to 12 Months',str(x)))


uncurated['matchez'].explode().value_counts()
uncurated[0:500]

In [261]:
# uncurated['clean']=uncurated['value'].apply(lambda x: f'["{x}"]')
# uncurated['clean']=uncurated['clean'].apply(lambda x: re.sub(r'"?\]"\]','"]',str(x))).apply(lambda x: re.sub(r'\["\["?','["',str(x))).apply(lambda x: re.sub(r'"\,(?!")','","',str(x)))

# uncurated['matchesss']=uncurated['clean'].apply(lambda x: re.sub(r'(?<![A-z])\s?(?<![A-z])\-\s?',' to ',str(x))).apply(lambda x: re.sub(r'2\-4T','["2T","3T","4T"]',str(x))).apply(lambda x: re.sub(r'2\-4Y','["2Y","3Y","4Y"]',str(x))).apply(lambda x: re.sub(r'newborn','Newborn',str(x))).apply(lambda x: re.sub(r'large','Large',str(x))).apply(lambda x: re.sub(r'medium','Medium',str(x))).apply(lambda x: re.sub(r'(?i)((?<=\d)M)|(Month(?!s))',' Months',str(x))).apply(lambda x: re.sub(r'(?i)(size\s?)|(\d+\-)','',str(x))).apply(lambda x: re.sub(r"(\s?\'\s?)",'"',str(x))).apply(lambda x: re.sub(r'Carhartt\® Shortall','',str(x))).apply(lambda x: re.sub(r'(?<=)Mns',' Months',str(x))).apply(lambda x: re.sub(r'\["\["','["',str(x))).apply(lambda x: re.sub(r'"\]"\]','"]',str(x))).apply(lambda x: re.sub(r'"?(?:\])?","(?:\[)"?','',str(x)))                                                                       
# import ast
# def remove_duplicates(A):
#     [A.pop(count) for count,elem in enumerate(A) if A.count(elem)!=1]
#     return A
# uncurated['matchez']=uncurated['matchesss'].apply(lambda x: remove_duplicates([n.strip() for n in ast.literal_eval(x)])).apply(lambda x: natsorted(x))
# uncurated['matchez']=uncurated['matchez'].apply(lambda x: re.sub(r"XL",'X-Large',str(x))).apply(lambda x: re.sub(r"\s?'\s?",'"',str(x))).apply(lambda x: re.sub(r'\[""\]','',str(x))).apply(lambda x: re.sub(r'one','One Size',str(x))).apply(lambda x: re.sub(r'  ',' ',str(x)))
# uncurated['matchez']=uncurated['matchez'].apply(lambda x: re.sub(r'18 Months","18 Months','18 Months',str(x))).apply(lambda x: re.sub(r'6 Months","6 to 9 Months','6 to 9 Months',str(x))).apply(lambda x: re.sub(r'(9 to 12 Months","12 Months)|(9 Months","9 to 12 Months)','9 to 12 Months',str(x))).apply(lambda x: re.sub(r'9 to 12 Months","12 Months','9 to 12 Months',str(x)))
# # uncurated[uncurated['matchez'].astype(str)=='["12 Months"]']['buckets'].explode().value_counts()
# uncurated=uncurated[uncurated['value'].astype(str)!='n/a']
# uncurated['matchez'].explode().value_counts()
# # uncurated[0:500]

In [256]:
uncurated['Q:size']=uncurated['matchesss'].apply(lambda x: re.sub(r'"\\"','"',str(x)))
match_uncurated=uncurated[['external_id','Q:size']]

In [257]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/BBBY --{get_df_name(matches)}_AI_size_curated.csv',index=False) 
    
looks_good('Bed Bath & Beyond', df, match_uncurated) 

#### Everyday/Dress Bodysuits

In [24]:
# size_na=size[size['value'].astype(str)=='n/a']
# everyday=size_na[(size_na['buckets'].astype(str)=='Everyday/Dress Bodysuits')]#&(size_na['buckets'].astype(str)=='Baby Bibs & Burp Cloths')&(size_na['buckets'].astype(str)=='T-Shirts')][0:500]
# trip=r'''(?i)(\d+(?: to |\s?\-\s?)\d+ (?:months?|years?))|((?:1[0-8]|(?<!\d)[1-9])(?:\.5)?(?:W|Y))|(\d+\s?Months?(?: and up)?)|([1-9]T)|(1[0-9]T)|(adult)|(infant)|((?:X.)?large)|(medium)|(newborn)|(one.?size)|(preemie)|(small)|(toddler)|(youth)|(X?X(?:L|S))|(Size.?(?:[1-9]|1[0-9])M)|(Size 6-12M)|(Carhartt.?.?Shortall)|(extra.?small)|()'''                                                              
# everyday['pajama']=everyday['product_name'].apply(lambda x: re_extract(trip,str(x)))
# everyday[everyday['pajama'].astype(str)!='[]']

### Pajama Jumpsuits

In [23]:
# size_na=size[size['value'].astype(str)=='n/a']
# everyday=size_na[(size_na['buckets'].astype(str)=='Pajama Jumpsuits')]#&(size_na['buckets'].astype(str)=='Baby Bibs & Burp Cloths')&(size_na['buckets'].astype(str)=='T-Shirts')][0:500]
# trip=r'''(?i)(\d+(?: to |\s?\-\s?)\d+ (?:months?|years?))|((?:1[0-8]|(?<!\d)[1-9])(?:\.5)?(?:W|Y))|(\d+\s?Months?(?: and up)?)|([1-9]T)|(1[0-9]T)|(adult)|(infant)|((?:X.)?large)|(medium)|(newborn)|(one.?size)|(preemie)|(small)|(toddler)|(youth)|(X?X(?:L|S))|(Size.?(?:[1-9]|1[0-9])M)|(Size 6-12M)|(Carhartt.?.?Shortall)|(extra.?small)|()'''                                                              
# everyday['pajama']=everyday['product_name'].apply(lambda x: re_extract(trip,str(x)))
# everyday[everyday['pajama'].astype(str)!='[]']

### Clothing & Accessory Variety Packs

In [212]:
size_na=size[size['value'].astype(str)=='n/a']
clothing=size_na[(size_na['buckets'].astype(str)=='Clothing & Accessory Variety Packs')]#&(size_na['buckets'].astype(str)=='Baby Bibs & Burp Cloths')&(size_na['buckets'].astype(str)=='T-Shirts')][0:500]
trip=r'''(?i)(\d+(?: to |\s?\-\s?)\d+ (?:months?|years?))|((?:1[0-8]|(?<!\d)[1-9])(?:\.5)?(?:W|Y))|(\d+\s?Months?(?: and up)?)|([1-9]T)|(1[0-9]T)|(adult)|(infant)|((?:X.)?large)|(medium)|(newborn)|(one.?size)|(preemie)|(small)|(toddler)|(youth)|(X?X(?:L|S))|(Size.?(?:[1-9]|1[0-9])M)|(Size 6-12M)|(Carhartt.?.?Shortall)|(extra.?small)|()'''                                                              
clothing['pajama']=clothing['product_name'].apply(lambda x: re_extract(trip,str(x)))

In [22]:
# clothing[clothing['pajama'].astype(str)!='[]']

In [17]:
# size[size['buckets'].astype(str)=='Clothing Sets & Variety Packs']['value'].explode().value_counts()

In [18]:
# size[size['buckets'].astype(str)=='Pajama Pants']['value'].explode().value_counts()

In [19]:
# size[size['buckets'].astype(str)=='Everyday/Dress Bodysuits']['value'].explode().value_counts()

In [20]:
# size[size['buckets'].astype(str)=='Pajama Jumpsuits']['value'].explode().value_counts()

In [21]:
# size['buckets'].explode().value_counts()

# AI Lumens

In [55]:
lumen=df[df['attribute'].astype(str)=='lumens']
# parameters
customer_id = '5'
formatted_attribute = 'Lumens'
customer_name='%bedbathandbeyond%'
buckets = """Office Chairs"""

attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')
value='%n/a%'
params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }

In [56]:
# Use KewWordProcessor to find a list of values for a single bucket 
df_strategy = query_from_file(file_name='../asset/easy.sql', params=params) #create query
list_values=df_strategy['value'].to_list()

kwp=KeywordProcessor()
## JUNK
# print(kwp.add_keywords_from_list(list_values))
# # Extract Key Words
# df['long_desc'].apply(lambda x: kwp.extract_keywords(x)) #

# df['matches'] = df['long_desc'].apply(lambda x: kwp.extract_keywords(x))
# df['matches'].explode().value_counts() 
list_values 

['0 - 449', '1000 - 1599', '1600+', '450 - 799', '800 - 999']

In [118]:
metric='lbs'
range_string = """
0 - 449
1000 - 1599
1600+
450 - 799
800 - 999
 """

range_params = {}
for range_entry in range_string.split('\n'):
    range_nums = re.findall('\d+', range_entry)
    if len(range_nums) > 0: 
        range_params[tuple(map(int, range_nums))] = range_entry.strip()


def  range_app(num_lst):
    updated_labels = []
    for num in num_lst:
        num = float(num)
        for range_param, range_label in range_params.items():
            if len(range_param) == 1:
                if num >= range_param[0]:
                    updated_labels.append(range_label)
            else:
                if num >= range_param[0] and num <= range_param[1]:
                    updated_labels.append(range_label)
    return updated_labels

In [123]:
lumens=lumen[lumen['value'].astype(str)!='n/a']
lumens['match']=lumens['value'].apply(lambda x: re.sub(r'','',str(x)))
pat=r'''(?i)(0\s?.\s?499)|(1000\s?.\s?1599)|(1600.)|(450\s?.\s?799)|(800\s?.\s?999)|(450\s?.\s?799)|((?<!\d)\d(?!\d))|()'''
lumens['matches']=lumens['match'].apply(lambda x: re_extract(pat,str(x)))
lumenz=lumens[(lumens['matches'].astype(str)=='[]')&(lumens['external_id'].astype(str)!='69805198')]
match=lumens[(lumens['matches'].astype(str)!='[]')&(lumens['external_id'].astype(str)!='69805198')]
match['Q:lumens']=''
match_match=match[['external_id','Q:lumens']]

lumenz['match']=lumenz['match'].apply(lambda x: re.sub(r'\,\s?','',str(x))).apply(lambda x: f"['{x}']")#.apply(lambda x: re.sub(r'\.\d+|\[|\]','',str(x)))
lumenz['match']=lumenz['match'].apply(lambda x: remove_duplicates([n.strip() for n in ast.literal_eval(x)])).apply(range_app).apply(lambda x: re.sub(r"'",'"',str(x)))
lumenz['match'].explode().value_counts()
lumenz['Q:lumens']=lumenz['match']
match_lumenz=lumenz[['external_id','Q:lumens']]

In [124]:
match_lumenz

,external_id,Q:lumens
1565380,67131389,"[""0 - 449""]"
1565381,67131402,"[""0 - 449""]"
1565382,65724705,"[""0 - 449""]"
1565383,47101609,"[""1000 - 1599""]"
1565384,45597435,"[""0 - 449""]"
...,...,...
9425751,65724439,"[""450 - 799""]"
9425752,47504691,"[""450 - 799""]"
9425753,47506336,"[""450 - 799""]"
9425755,65724637,"[""800 - 999""]"


In [125]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/BBBY --{get_df_name(matches)}-lumens.csv',index=False) 
    
looks_good('Bed Bath & Beyond', df, match_lumenz) 

# AI 432

In [30]:
feats=df[(df['attribute'].astype(str)=='features')&((df['buckets'].astype(str)=='Dining Chairs')|(df['buckets'].astype(str)=='Benches')|(df['buckets'].astype(str)=='Coffee Tables')|(df['buckets'].astype(str)=='Dining Tables'))]
print(len(feats))

3269


In [34]:
pat='''(?i)|(non.marketing)|(floor.?protector)|(stain.?resistant)|(removable.?cushion)|(removable.?cover)|(swivel)|(tuft(ed|ing))|(nailhead.?trim)|(leveler)|(adjustable)|()'''
feats['match']=feats['long_desc'].apply(lambda x: re_extract(pat,str(x)))

In [39]:
x=feats[(feats['match'].astype(str)!='[]')]
y=feats[(feats['value'].astype(str)=='n/a')]
print(len(x))
print(len(y))

138
1532


In [41]:
x['Q:features']=''
y['Q:features']=''
match_features_x=x[['external_id','Q:features']]
match_features_y=y[['external_id','Q:features']]

In [44]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/BBBY --{get_df_name(matches)}-Features_AI432_y.csv',index=False) 
    
looks_good('Bed Bath & Beyond', df, match_features_y) 

# AI Bed/Bedding Size

In [6]:
bed=df[df['attribute'].astype(str)=='bed_bedding_size']
pat='''(?i)(futon)|(pull.?out)|(sleeper.?sofa)|()'''
bed['match']=bed['long_desc'].apply(lambda x: re_extract(pat,str(x)))

In [26]:
beds=bed[(bed['match'].astype(str)!='[]')&(bed['value'].astype(str)=='n/a')]
beds['match']=beds['match'].apply(lambda x: str(x).title()).apply(lambda x: re.sub(r'\-',' ',str(x))).apply(lambda x: re.sub(r'Pullout','Pull Out',str(x))).apply(lambda x: re.sub(r'(Pull Out)|(Sleeper Sofa)','Sofa',str(x)))
import ast
def remove_duplicates(A):
    [A.pop(count) for count,elem in enumerate(A) if A.count(elem)!=1]
    return A
beds['match']=beds['match'].apply(lambda x: remove_duplicates([n.strip() for n in ast.literal_eval(x)]))
beds['match'].explode().value_counts()

Futon    156
Sofa      66
Name: match, dtype: int64

In [27]:
beds['Q:bed_bedding_size']=beds['match'].apply(lambda x: re.sub(r"'",'"',str(x)))
match_beds=beds[['external_id','Q:bed_bedding_size']]

In [29]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/BBBY --{get_df_name(matches)}-bed.csv',index=False) 
    
looks_good('Bed Bath & Beyond', df, match_beds) 

# AI Light Bulbs

In [22]:
bulbs=df[(df['buckets'].astype(str)=='Light Bulbs')&((df['attribute'].astype(str)=='bulb_color')|(df['attribute'].astype(str)=='bulb_type')|(df['attribute'].astype(str)=='certifications')|(df['attribute'].astype(str)=='recommended_use')|(df['attribute'].astype(str)=='wattage'))]      

In [35]:
bulb_color=bulbs[(bulbs['attribute'].astype(str)=='bulb_color')&(bulbs['value'].astype(str)=='n/a')]
certs='''(?i)(day.?light)|(warm.?white)|(cool.?white)|(amber)|(soft.?white)|(bright white light)|()'''
bulb_color['match']=bulb_color['long_desc'].apply(lambda x: re_extract(certs,str(x)))
bulb_color['match']=bulb_color['match'].apply(lambda x: str(x).title())
import ast
def remove_duplicates(A):
    [A.pop(count) for count,elem in enumerate(A) if A.count(elem)!=1]
    return A
bulb_color['match']=bulb_color['match'].apply(lambda x: remove_duplicates([n.strip() for n in ast.literal_eval(x)]))
bulb_color['match']=bulb_color['match'].apply(lambda x: re.sub(r"'",'"',str(x))).apply(lambda x: re.sub(r'(?i)(bright.?white.?light)','Warm White',str(x))).apply(lambda x: re.sub(r'(?i)(Amber)','Daylight',str(x))).apply(lambda x: re.sub(r'(?i)(soft.?white(?:.light)?)','Cool White',str(x)))
bulbz=bulb_color[bulb_color['match'].astype(str)!='[]']
print(len(bulbz))
bulbz['Q:bulb_color']=bulbz['match']
match_bulbz=bulbz[['external_id','Q:bulb_color']]


60


In [36]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/BBBY --{get_df_name(matches)}-recommended_use.csv',index=False) 
    
looks_good('Bed Bath & Beyond', df, match_bulbz) 

In [44]:
bulb_type=bulbs[(bulbs['attribute'].astype(str)=='bulb_type')&(bulbs['value'].astype(str)=='n/a')]
certs='''(?i)(fluorescent)|(halogen)|(incandescent)|(\bled\b)|()'''
bulb_type['match']=bulb_type['long_desc'].apply(lambda x: re_extract(certs,str(x)))
# types=bulb_type[bulb_type['match'].astype(str)!='[]']
# types['Q:bulb_type']=''
# match_bulbs=types[['external_id','Q:bulb_type']]

bulb_type[bulb_type['match'].astype(str)!='[]']

,attribute,buckets,bucket_id,value,customer_name,external_id,product_name,long_desc,curated_date,resolution,curation_tasks.curated_by,match


In [43]:
# cert=bulbs[bulbs['attribute'].astype(str)=='certifications']
# certs='''(?i)(ce.?marketing)|(dlc)|(energy.?star)|(etl)|(fcc)|(rohs)|(\bUL\b)|()'''
# cert['match']=cert['long_desc'].apply(lambda x: re_extract(certs,str(x)))
# certs=cert[cert['match'].astype(str)!='[]']
# certs['Q:certifications']='' 
# match_cert=certs[['external_id','Q:certifications']]
# match_cert

In [ ]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/BBBY --{get_df_name(matches)}-recommended_use.csv',index=False) 
    
looks_good('Bed Bath & Beyond', df, match_cert) 

In [40]:
wat=bulbs[bulbs['attribute'].astype(str)=='wattage']                                                   
wat_na=bulbs[(bulbs['attribute'].astype(str)=='wattage')&(bulbs['value'].astype(str)=='n/a')]

rec=bulbs[bulbs['attribute'].astype(str)=='recommended_use']
rec_na=bulbs[(bulbs['attribute'].astype(str)=='recommended_use')&(bulbs['value'].astype(str)=='n/a')]

cert=bulbs[bulbs['attribute'].astype(str)=='certifications']
cert_na=bulbs[(bulbs['attribute'].astype(str)=='certifications')&(bulbs['value'].astype(str)=='n/a')]

types=bulbs[bulbs['attribute'].astype(str)=='bulb_type']
types_na=bulbs[(bulbs['attribute'].astype(str)=='bulb_type')&(bulbs['value'].astype(str)=='n/a')]

colors=bulbs[bulbs['attribute'].astype(str)=='bulb_color']
colors_na=bulbs[(bulbs['attribute'].astype(str)=='bulb_color')&(bulbs['value'].astype(str)=='n/a')]

print(len(wat_na)/len(wat)*100) # Not a lot of N/A values
print(len(cert_na)/len(cert)*100)  # Wipe/Rules
print(len(types_na)/len(types)*100) # Low # N/A-good
print(len(rec_na)/len(rec)*100)   # Wipe/Rules
print(len(colors_na)/len(colors)*100)

6.209150326797386
67.83216783216784
18.627450980392158
83.56164383561644
62.12624584717608


In [42]:
# light='''(?i)(lamp)|(pendant.?lighting)|(track.{0,10}light)|(recessed.?lights)|(night lights)|(under cabinet.?lights)|(flush.{0,7}light)|(semi.?flush.{0,7}light)|(rope.?light)|(string.?light)|(sconces)|(vanity.?light)|()'''   
# rec['match']=rec['long_desc'].apply(lambda x: re_extract(light,str(x)))
# rec_df=rec[rec['match'].astype(str)!='[]']
# rec_df['Q:recommended_use']=''
# match_rec=rec_df[['external_id','Q:recommended_use']]
# match_rec

,external_id,Q:recommended_use


In [43]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/BBBY --{get_df_name(matches)}-recommended_use.csv',index=False) 
    
looks_good('Bed Bath & Beyond', df, match_rec) 

In [ ]:
# Waiting on Ross #477 or 478

In [64]:
feat=df[(df['attribute'].astype(str)=='features')&((df['buckets'].astype(str)=='Coffee Tables')|(df['buckets'].astype(str)=='Dining tables')|(df['buckets'].astype(str)=='Chairs')|(df['buckets'].astype(str)=='Furniture variety packs')|(df['buckets'].astype(str)=='Serving/Rolling carts'))]
feat['Q:features']=''
match_feat=feat[['external_id','Q:features']]

In [ ]:
# Waiting on Ross #477 or 478

In [59]:
mat=df[(df['attribute'].astype(str)=='material')&(df['value'].astype(str)=='Fabric')]
mat['value'].explode().value_counts()
mat['Q:material']=''
match_mat=mat[['external_id','Q:material']]

In [65]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/BBBY --{get_df_name(matches)}-features.csv',index=False) 
    
looks_good('Bed Bath & Beyond', df, match_feat) 

In [66]:
# Waiting on Jeff to approve adding value Futon and Pull Out to values AI 455


In [21]:
bed=df[(df['attribute'].astype(str)=='bed_bedding_size')&(df['buckets'].astype(str)=='Sleeper Sofas/Sofa Beds/Futons')&(bed['value'].astype(str)=='n/a')]
print(len(bed))

197


In [12]:
# bed['value'].explode().value_counts()
# bed[bed['value'].astype(str)=='n/a']

# sofa=r'''((?:standard|twin(?:.?xl)?|split(?!.back)|\bfull\b|queen|(?:California.?)?\bKing).{0,7}bed)|(futon)|()|()'''                                                              
# bed['match']=bed['long_desc'].apply(lambda x: re_extract(sofa,str(x)))
# sofa=bed[bed['match'].astype(str)=='[]']
# print(len(sofa))
# sofa

In [ ]:
# Waiting on Ross and Jeff to meet with client and get OK

In [ ]:
# Clothing Sets & Variety Packs-Pajamas/Kids --> one size fits all

In [142]:
size_na=df[(df['attribute'].astype(str)=='size')&(df['value'].astype(str)=='n/a')]
size_na['buckets'].explode().value_counts().reset_index()['index'].to_list()

['Clothing Sets & Variety Packs',
 'Baby Bibs & Burp Cloths',
 'T-Shirts',
 'Clothing & Accessory Variety Packs',
 'Pajama Jumpsuits',
 'Role Playing Toys',
 'Full Brim Hats',
 'Baby Feeding Bottles & Accessories',
 'Slacks/Pants',
 'Bras',
 'Feminine Hygiene',
 'Leggings',
 'Non-Brim Hats',
 'Socks',
 'Everyday/Dress Bodysuits',
 'Robes',
 'Baby Food',
 'Jackets/Coats',
 'Swim Variety Packs',
 'Tops',
 'Dresses & Gowns',
 'Pet Bowls & Feeders',
 'Baby Jumpers, Swings, & Rockers',
 'Everyday/Dress Jumpsuits',
 'Bassinets & Cradles',
 'Button-Downs',
 'Sweatshirts/Fleece Pullovers',
 'Pantyhose/Tights',
 'Everyday/Dress Shorts',
 'Baby Safety Monitors',
 'Swim Bottoms',
 'Headbands',
 'Protective Full Body Swim',
 'Pet Houses, Crates, & Kennels',
 'Pet Car Accessories',
 'One-Piece Swimsuits',
 'Accessories Variety Packs',
 'Overalls',
 'Everyday/Dress Footwear',
 'Helmets',
 'Sweaters',
 'Underwear',
 'Night Dresses & Shirts',
 'Protective/Active Bodysuits',
 'Pet Beds & Cots',
 'Corse

In [135]:
clothing=size_na[(size_na['buckets'].astype(str)=='Clothing & Accessory Variety Packs')]#&(size_na['buckets'].astype(str)=='Baby Bibs & Burp Cloths')&(size_na['buckets'].astype(str)=='T-Shirts')][0:500]
trip=r'''(?i)(\d+(?: to |\s?\-\s?)\d+ (?:months?|years?))|((?:1[0-8]|(?<!\d)[1-9])(?:\.5)?(?:W|Y))|(\d+\s?Months?(?: and up)?)|([1-9]T)|(1[0-9]T)|(adult)|(infant)|((?:X.)?large)|(medium)|(newborn)|(one.?size)|(preemie)|(small)|(toddler)|(youth)|(X?X(?:L|S))|(Size.?(?:[1-9]|1[0-9])M)|(Size 6-12M)|(Carhartt.?.?Shortall)|()'''                                                              
clothing['pajama']=clothing['product_name'].apply(lambda x: re_extract(trip,str(x)))

In [15]:
# clothing['pajama'].explode().value_counts()

# # 2T, 3T, 4T, 5T, 6T,  8T, Toddler, Small, Infant, XS, Newborn, Medium, Large, XXL, X-Large, Adult,  4Y, 5Y, 6Y, 7Y, 8Y, 9Y, 10Y, 12Y, 14Y,        4M, 3M, 9M, 6M, 18M, 24M, 12M, Pajamas, Carhart Shorttall, 


In [14]:
# print('No Matches: '+str(len(clothing[clothing['pajama'].astype(str)=='[]'])))
# print('Matches: '+str(len(clothing[clothing['pajama'].astype(str)!='[]'])))
# clothing[clothing['pajama'].astype(str)!='[]'][0:500]

In [69]:
feature_na=df[(df['attribute'].astype(str)=='features')&(df['value'].astype(str)=='n/a')]
print(len(feature_na))
sofa=feature_na[(feature_na['buckets'].astype(str)=='Sleeper Sofas/Sofa Beds/Futons')|(feature_na['buckets'].astype(str)=='Sofas/Couches/Loveseats/Settees ')]
print(len(sofa))

395797
52


In [3]:
depth_sheets=df[(df['attribute'].astype(str)=='depth_sheets')&(df['buckets'].astype(str)=='Bedding Variety Packs')]
print('Depth (Sheets) Length: '+str(len(depth_sheets)))
depth_na=depth_sheets[depth_sheets['value'].astype(str)=='n/a']
print('Number N/A: '+str(len(depth_na)))
# depth_na[0:500]

Depth (Sheets) Length: 34430
Number N/A: 24932


In [4]:
pat=r'''(?i)(?i)(fitted sheets? fits? mattresses up to \d+\\?"?)|((?<!\/)\d+\\"\s?D(?:eep|epth)?\b)|()'''
trip=r'''(?i)((?:\d+(?:(?:\-?\s?\d+\/\d+)?(?:\.\d+)?)?|\d+\/\d+)\\{0,2}(?:(?:")?(?:”)?(?:'{0,2})?(?:\W?inc?h?e?s?)?(?:\W?fo?o?e?e?t)?(?:yds)?(?:\s?[lwhd]?))\\{0,2}(?:(?:")?(?:”)?(?:'{0,2})?(?:\W?inc?h?e?s?)?(?:\W?fo?o?e?e?t)?(?:yds)?(?:\s?[lwhd]?)).?x.?(?:\d+(?:(?:\-?\s?\d+\/\d+)?(?:\.\d+)?)?|(\d+\/\d+))\\{0,2}(?:(?:")?(?:”)?(?:'{0,2})?(?:\W?inc?h?e?s?)?(?:\W?fo?o?e?e?t)?(?:yds)?(?:\s?[lwhd]?))\\{0,2}(?:(?:")?(?:”)?(?:'{0,2})?(?:\W?inc?h?e?s?)?(?:\W?fo?o?e?e?t)?(?:yds)?(?:\s?[lwhd]?)).?x.?(?:\d+(?:(?:\-?\s?\d+\/\d+)?(?:\.\d+)?)?|(\d+\/\d+))\\{0,2}(?:(?:")?(?:”)?(?:'{0,2})?(?:\W?inc?h?e?s?)?(?:\W?fo?o?e?e?t)?(?:yds)?(?:\s?[lwhd]?))\\{0,2}(?:(?:")?(?:”)?(?:'{0,2})?(?:\W?inc?h?e?s?)?(?:\W?fo?o?e?e?t)?(?:yds)?(?:\s?[lwhd]?)))|()'''
fitted_sheet='fitted.?sheet'
depth_na['fitted']=depth_na['long_desc'].apply(lambda x: re_extract(pat,str(x)))
depth_na['fitted_sheet']=depth_na['long_desc'].apply(lambda x: re_extract(fitted_sheet,str(x)))
fitted=depth_na[(depth_na['fitted'].astype(str)!='[]')&(depth_na['fitted_sheet'].astype(str)!='[]')]
print(len(fitted))

3


In [44]:
fitted['Q:depth_sheets']=fitted['fitted'].apply(lambda x: re.sub(r'[A-z]|\s','',str(x))).apply(lambda x: re.sub(r'\\?"',' in',str(x))).apply(lambda x: re.sub(r"'",'"',str(x))).apply(lambda x: f'[{x}]').apply(lambda x: re.sub(r'(?<=\d)"]',' in"]',str(x)))
fitted['Q:depth_sheets'].explode().value_counts()
match_fitted=fitted[['external_id','Q:depth_sheets']]

In [45]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/BBBY --{get_df_name(matches)}-AI_match_fitted.csv',index=False) 
    
looks_good('Bed Bath & Beyond', df, match_fitted) 

In [82]:
depth_na['trip']=depth_na['long_desc'].apply(lambda x: re_extract(trip,str(x)))
trips=fitted[fitted['trip'].astype(str)!='[]']

In [84]:
pattern='(?i)((?:\d+\.)?\d+\\?"\s?D)|((?<=\d\d\\" \w \d\d\\" \w )(?<=)(?:0\.)?\d+\\?")|((?<=\d\d\\" \w x \d\d\\" \w x )(?:0\.)?\d+\\?"(?! H))|(\d+\\?"\s?D)|()'
trips['trips']=trips['trip'].apply(lambda x: re_extract(pattern,str(x)))
# trips['trips'].explode().value_counts()
trips['Q:depth_sheets']=''
match_trips=trips[['external_id','Q:depth_sheets']]

In [68]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/BBBY --{get_df_name(matches)}-AI_match_trips.csv',index=False) 
    
looks_good('Bed Bath & Beyond', df, match_trips) 

In [ ]:
trips['trips'].apply(lambda x: re.sub(r'\\?"\s?D',' in',str(x)))

In [13]:
# features=df[(df['attribute'].astype(str)=='features')]
# pat=r'(?i)(sofa)|(couch)|()'
# features['pillow']=features['product_name'].apply(lambda x: re_extract(pat,str(x)))


# na=features[(features['pillow'].astype(str)=='[]')]
# na['Q:features']=''
# matchna=na[['external_id','Q:features']]
# na[0:500]

In [16]:
# rugs=r'''(3'6\\?"? x 5'6\\?"?)|()'''
# df['rugs']=df['product_name'].apply(lambda x: re_extract(rugs,str(x)))
# three=df[(df['rugs'].astype(str)!='[]')&(df['value'].astype(str)=='n/a')&(df['attribute'].astype(str)=='rug_size')]
# three['Q:rug_size']="4' x 6'"
# three

In [24]:
match_three=three[['external_id','Q:rug_size']]
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/BBBY --{get_df_name(matches)}-AI.csv',index=False) 
    
looks_good('Bed Bath & Beyond', df, match_three) 

In [ ]:
pat=r'(?i)(from.{0,20}solid)|(add color.{0,20}solid)|()'
shade=r'(?i)(shade)|(blind|)|(Curtain)|(shutter)|(blockout)|()'
solid_name=r'(?i)(solid.{0,20}flannel)|()'

pattern=df[(df['attribute'].astype(str)=='pattern')&(df['value'].astype(str)=='Solid')&(df['resolution'].astype(str)=='rules')]
pattern['pat']=pattern['long_desc'].apply(lambda x: re_extract(pat,str(x)))
pattern['shade']=pattern['product_name'].apply(lambda x: re_extract(solid_name,str(x)))

In [ ]:
x=pattern[(pattern['shade'].astype(str)=='[]')&(pattern['buckets'].astype(str)!='Curtains')&(pattern['buckets'].astype(str)!='Towels Variety Packs')&(pattern['buckets'].astype(str)!='Washcloths')&(pattern['value'].astype(str)!='n/a')]
a=x[x['buckets'].astype(str)=='Throw/Decorative Pillows']
print(len(a))
a[0:500]
a['Q:pattern']=''
match=a[['external_id','Q:pattern']]
print(len(match))

In [ ]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/BBBY --{get_df_name(matches)}-AI.csv',index=False) 
    
looks_good('Bed Bath & Beyond', df, match) 

In [ ]:
weave=df[(df['attribute'].astype(str)=='weave')&(df['value'].astype(str)=='n/a')]
print(len(weave))
pat=r'(?i)((?:Two|Single).?Ply)|(Twill)|(Tufted)|(Shag)|(Satin)|(Sateen)|(Quilt(?:ed)?)|(Pinpoint)|(Percale)|(Oxford)|(Microfiber)|(Machine.?(?:Made|woven))|(Knotted)|(Jersey)|(Jacquard)|(Interlock)|(Hooked)|(Flatweave)|(Flannel)|(Dobby)|(Damask)|()'
weave['match']=weave['long_desc'].apply(lambda x: re_extract(pat,str(x)))
weave['match_name']=weave['product_name'].apply(lambda x: re_extract(pat,str(x)))

In [ ]:
curate=weave[(weave['match'].astype(str)!='[]')]
print(len(curate))
curate['matches']=curate['match'].astype(str).apply(lambda x: re.sub(r'(?i)Quilt(?:ed)?','Quilted',str(x))).apply(lambda x: re.sub(r'(?i)(Machine.?(?:Made|woven))','Machine Made',str(x))).apply(lambda x: re.sub(r"'",'"',str(x))).apply(lambda x: x.title())
curate['matches']=curate['matches'].apply(lambda x: re.sub(r'(?i)\-Ply',' Ply',str(x)))
# curate['matches'].explode().value_counts()

In [ ]:
import ast
def remove_duplicates(A):
    [A.pop(count) for count,elem in enumerate(A) if A.count(elem)!=1]
    return A
curate['matchez']=curate['matches'].apply(lambda x: remove_duplicates([n.strip() for n in ast.literal_eval(x)])).apply(lambda x: natsorted(x))

In [ ]:
curate['Q:weave']=curate['matchez'].apply(lambda x: re.sub(r"'",'"',str(x)))

In [ ]:

curate['Q:weave']=curate['Q:weave'].apply(lambda x: re.sub(r'\\|Xa0',' ',str(x))).apply(lambda x: re.sub(r'\, "',',"',str(x)))
curate['Q:weave'].explode().value_counts()
match=curate[['external_id','Q:weave']]

In [ ]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/BBBY --{get_df_name(matches)}-AI.csv',index=False) 
    
looks_good('Bed Bath & Beyond', df, match) 

In [ ]:
tc=df[df['attribute'].astype(str)=='thread_count']
pat=r'(?i)(sheet)|()'
tc['sheet']=tc['product_name'].apply(lambda x: re_extract(pat, str(x)))
sheet=tc[(tc['sheet'].astype(str)!='[]')&(tc['value'].astype(str)=='n/a')]

pat=r'(?i)(thread.?count)|(\d+TC)|()'
sheet['thread_count']=sheet['long_desc'].apply(lambda x: re_extract(pat, str(x)))
print(len(sheet[sheet['thread_count'].astype(str)!='[]']))
na=sheet[sheet['thread_count'].astype(str)!='[]']
na['Q:thread_count']=''
match_na=na[['external_id','Q:thread_count']]

In [ ]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/BBBY-{get_df_name(matches)}-matches.csv',index=False) 
    
looks_good('Bed Bath & Beyond', df, match_na)

In [ ]:
pat=r'(?i)(flannel)|(jersey)|()'
sheet['match']=sheet['long_desc'].apply(lambda x: re_extract(pat, str(x)))

In [ ]:
# tc[(tc['sheet'].astype(str)!='[]')&(tc['value'].astype(str)!='n/a')]

In [ ]:
x=sheet[(sheet['thread_count'].astype(str)=='[]')&(sheet['match'].astype(str)=='[]')]#['buckets'].explode().value_counts()
y=x[(x['buckets'].astype(str)!='Pillowcases & Pillow Protectors')&(x['buckets'].astype(str)!='Mattress Accessories & Specialty Protectors')&(x['buckets'].astype(str)!='Comforters')]
print(len(y))
y['buckets'].explode().value_counts()
# y[0:500]

In [ ]:

tc_na=tc[tc['value'].astype(str)=='n/a']

pat=r'(?i)(flannel)|(jersey)|()'
tc_na['match']=tc_na['long_desc'].apply(lambda x: re_extract(pat, str(x)))

pat=r'(?i)(pillow)|()'
tc_na['name']=tc_na['product_name'].apply(lambda x: re_extract(pat, str(x)))

count=tc_na[(tc_na['match'].astype(str)=='[]')&(tc_na['name'].astype(str)=='[]')&((tc_na['buckets'].astype(str)=='Bedding Variety Packs')|(tc_na['buckets'].astype(str)=='Bed Sheets'))]#[0:500]

In [ ]:
pat=r'(?i)(thread.?count)|()'
count['thread_count']=count['long_desc'].apply(lambda x: re_extract(pat, str(x)))
count_na=count[count['thread_count'].astype(str)!='[]']
print(len(count_na))
count_na['Q:thread_count']=''
match_count=count_na[['external_id','Q:thread_count']]

def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/BBBY-{get_df_name(matches)}-matches.csv',index=False) 
    
looks_good('Bed Bath & Beyond', df, match_count)

In [ ]:
pat=r'(?i)(solid)|()'
df['solid']=df['value'].apply(lambda x: re_extract(pat, str(x)))

In [ ]:
pat=r'(?i)Cabin Print'
df['match']=df['long_desc'].apply(lambda x: re_extract(pat, str(x)))
micro=x[x['match'].astype(str)!='[]']
print(len(micro))
micro

In [ ]:
x=df[(df['attribute'].astype(str)=='pattern')&(df['solid'].astype(str)!='[]')&(df['resolution'].astype(str)=='rules')&(df['buckets'].astype(str)!='Window Blinds')]

In [ ]:
# bed=x[x['buckets'].astype(str)=='Bedding Variety Packs']
# pat=r'(?i)(.{0,20}solid.{0,20})|()'
# reverse=r'(?i)(Reverses to matching solid design)|()'
# bed['match']=bed['long_desc'].apply(lambda x: re_extract(pat, str(x)))
# bed['reverse']=bed['long_desc'].apply(lambda x: re_extract(reverse, str(x)))
# bed[(bed['match'].astype(str)!='[]')][0:500]

In [ ]:
# x['buckets'].explode().value_counts()

In [ ]:
x=df[(df['attribute'].astype(str)=='color')&((df['value'].astype(str)=='Assorted')|(df['value'].astype(str)=='assorted')|(df['value'].astype(str)=='Assorted Colors'))]
pat=r'(?i)((?<!sticker )sheet)|()'
x['match']=x['long_desc'].apply(lambda x: re_extract(pat, str(x)))
x['match_name']=x['product_name'].apply(lambda x: re_extract(pat, str(x)))

In [ ]:
len(x[(x['match'].astype(str)!='[]')&(x['match_name'].astype(str)!='[]')])
sheet=x[(x['match'].astype(str)!='[]')&(x['match_name'].astype(str)!='[]')]
sheet['na']=''
match_na=sheet[['external_id','na']]
print(len(match_na))


def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/BBBY --{get_df_name(matches)}-AI.csv',index=False) 
    
looks_good('Bed Bath & Beyond', df, match_na) 

In [ ]:
# df[((df['bucket_id']==7309)|(df['bucket_id']==6630)|(df['bucket_id']==8131)|(df['bucket_id']==6714)|(df['bucket_id']==6670)|(df['bucket_id']==8705)|(df['bucket_id']==8823)|(df['bucket_id']==9031)|(df['bucket_id']==7452)|(df['bucket_id']==8975)|(df['bucket_id']==9668)|(df['bucket_id']==6706)|(df['bucket_id']==9143)|(df['bucket_id']==7252)|(df['bucket_id']==9268)|(df['bucket_id']==8395)|(df['bucket_id']==9667)|(df['bucket_id']==9403)|(df['bucket_id']==7906)|(df['bucket_id']==7085)|(df['bucket_id']==8670)|(df['bucket_id']==7969)|(df['bucket_id']==8958)|(df['bucket_id']==8831)|(df['bucket_id']==7930))&((df['attribute'].astype(str)=='care'))]['buckets'].explode().value_counts()                             

In [ ]:
# print(len(df[(df['attribute'].astype(str)=='care')&(df['value'].astype(str)!='n/a')]['bucket_id'].explode().value_counts().reset_index().sort_values('index')['index'].to_list()))
# df[(df['attribute'].astype(str)=='care')&(df['value'].astype(str)!='n/a')]['bucket_id'].explode().value_counts().reset_index().sort_values('index')['index'].to_list()[0:500]

In [ ]:
# df[(df['attribute']=='certifications')&(df['resolution']!='rules')]

In [ ]:
# x=df[(df['attribute'].astype(str)=='pattern')&(df['value'].astype(str)=='Damask')&(df['resolution'].astype(str)=='rules')]
# x['Q:pattern']='n/a'
# match=x[['external_id','Q:pattern']]
# match

In [ ]:
# x=df[(df['attribute'].astype(str)=='personalization_type')&(df['value'].astype(str)=='Text')]
# x['Q:personalization_type']='n/a'
# match=x[['external_id','Q:personalization_type']]
# print(len(match))
# match
# print(len(x))
# x[0:500]

In [ ]:
# def get_df_name(df):
#     name =[x for x in globals() if globals()[x] is df][1]
#     return name

# def looks_good(customer, df, matches): 
#     drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
# #     non_matches = df[df['matches'].astype(str) == '[]']
# #     non_matches[curation_col] = r'n/a'
# #     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
#     matches.to_csv(f'{drive_path}/BBBY --{get_df_name(matches)}-matches-personalization.csv',index=False) 
    
# looks_good('Bed Bath & Beyond', df, match)

In [ ]:
# att=df[df['attribute'].astype(str)=='certifications']
# print('Length of df: '+str(len(att)))
# print('Top 5 Resolutions: '+str(att['resolution'].explode().value_counts().reset_index()['index'].to_list()[0:5]))

In [ ]:
# res=df[df['attribute'].astype(str)=='certifications']#['resolution'].explode().value_counts()
# print(len(res))
# res[res['resolution'].astype(str)=='standard'].sort_values('curated_date',ascending=False)[0:500]
# res[res['resolution'].astype(str)=='standard']['value'].explode().value_counts()#
# df[df['attribute'].astype(str)=='battery_type']

In [ ]:
# x=df[(df['attribute'].astype(str)=='care')]#&(df['value'].astype(str)=='UL Listed')]
# print(len(x))
# x['value'].explode().value_counts()
# # x[0:500]
# # x[x['value'].astype(str)=='All Types']
# # x[x['value'].str.contains('All', na = False)]
# # x[x['value'].astype(str)=='Natural']['long_desc']
# na=x[x['value'].astype(str)=='n/a']
# na['buckets'].explode().value_counts()
# df=df[df['attribute']=='care']

In [ ]:
# df['attribute'].explode().value_counts()[0:100]

In [ ]:
# df=df[df['attribute'].astype(str)!='keywords']
# print('Total SKUs not counting Keywords: '+str(len(df)))
# df['attribute'].explode().value_counts()[0:20]

In [ ]:
# na=df[df['value'].astype(str)=='n/a']
# print('Total N/A values: '+str(len(na)))
# na['attribute'].explode().value_counts()[0:20]

### attribute

In [ ]:
# z={}
# for idx, name in enumerate(na['attribute'].explode().value_counts()[0:50].index.tolist()):
#     z[name]=na['attribute'].value_counts()[idx]
# data_items = z.items()
# data_list = list(data_items)
# u=pd.DataFrame(data_list)
# u.rename(columns={ u.columns[0]: "Attribute",u.columns[1]: "N/A Count" }, inplace = True)

In [ ]:
# x={}
# for idx, name in enumerate(df['attribute'].explode().value_counts()[0:50].index.tolist()):
#     x[name]=df['attribute'].value_counts()[idx]

# data_items = x.items()
# data_list = list(data_items)
# y=pd.DataFrame(data_list)
# y.rename(columns={ y.columns[0]: "Attribute",y.columns[1]: "Total Count" }, inplace = True)

In [ ]:
# merge=y.merge(u,how='outer',left_on=['Attribute'],right_on=['Attribute'])
# merge['percent']=round(merge['N/A Count']/merge['Total Count']*100,2)
# merge=merge[merge['percent'].astype(str)!='nan']
# merge=merge.sort_values('percent',ascending=False)

In [ ]:
# over_10k=merge[merge['Total Count']>100]
# over_10k.sort_values('Total Count',ascending=False)

In [ ]:
# top_merge=merge[merge['percent'].astype(str)>'30'].reset_index(drop=True)
# top_merge#.sort_values('Total Count',ascending=False)

In [ ]:
# for i in range(len(top_merge)):
#     print('                                                                 ')
#     print('Attribute: '+str(top_merge['Attribute'][i].upper())+'________ Percent N/A:'+str(top_merge['percent'][i]))
#     print('Number of N/A Values: '+str(top_merge['Total Count'][i]))
#     print(df[df['attribute'].astype(str)==f"{top_merge['Attribute'][i]}"]['buckets'].explode().value_counts()[0:20])
#     print('#############################################################')
#     print('      

In [ ]:
# def na_analysis(date1,date2):
#     print('Start Date: '+str(date1)+' End Date: '+str(date2))
#     x=df[(df['curated_date'].astype(str)>=date1)&(df['curated_date'].astype(str)<=date2)]
#     print('Total curated on this day: '+str(len(x)))
#     nine_thirteen=na[(na['curated_date'].astype(str)>=date1)&(na['curated_date'].astype(str)<=date2)]
#     print('Total N/A values this day:'+str(len(nine_thirteen)))
#     print('Percent N/A: '+str(round((len(nine_thirteen))/(len(x))*100,2))+ '%')
#     print(nine_thirteen['attribute'].explode().value_counts()[0:10])

In [ ]:
# Buckets for an attribute

In [ ]:
# z={}
# for idx, name in enumerate(na['buckets'].explode().value_counts()[0:50].index.tolist()):
#     z[name]=na['buckets'].value_counts()[idx]
# data_items = z.items()
# data_list = list(data_items)
# u=pd.DataFrame(data_list)
# u.rename(columns={ u.columns[0]: "Buckets",u.columns[1]: "N/A Count" }, inplace = True)

In [ ]:
# x={}
# for idx, name in enumerate(df['buckets'].explode().value_counts()[0:50].index.tolist()):
#     x[name]=df['buckets'].value_counts()[idx]

# data_items = x.items()
# data_list = list(data_items)
# y=pd.DataFrame(data_list)
# y.rename(columns={ y.columns[0]: "Buckets",y.columns[1]: "Total Count" }, inplace = True)

In [ ]:
# merge=y.merge(u,how='outer',left_on=['Buckets'],right_on=['Buckets'])
# merge['percent']=round(merge['N/A Count']/merge['Total Count']*100,2)
# merge=merge[merge['percent'].astype(str)!='nan']
# merge=merge.sort_values('percent',ascending=False)

In [ ]:
# over_10k=merge[merge['Total Count']>100]
# over_10k.sort_values('Total Count',ascending=False)

In [ ]:
# top_merge=merge[merge['percent'].astype(str)>'30'].reset_index(drop=True)
# top_merge#.sort_values('Total Count',ascending=False)

In [ ]:
# top_merge['N/A Count'].sum()

In [ ]:
# for i in range(len(top_merge)):
#     print('                                                                 ')
#     print('Buckets: '+str(top_merge['Buckets'][i].upper())+'________ Percent N/A:'+str(top_merge['percent'][i]))
#     print('Number of N/A Values: '+str(top_merge['Total Count'][i]))
#     print(df[df['buckets'].astype(str)==f"{top_merge['Buckets'][i]}"]['buckets'].explode().value_counts()[0:20])
#     print('#############################################################')
#     print('                                                                 ')

In [ ]:
# def na_analysis(date1,date2):
#     print('Start Date: '+str(date1)+' End Date: '+str(date2))
#     x=df[(df['curated_date'].astype(str)>=date1)&(df['curated_date'].astype(str)<=date2)]
#     print('Total curated on this day: '+str(len(x)))
#     nine_thirteen=na[(na['curated_date'].astype(str)>=date1)&(na['curated_date'].astype(str)<=date2)]
#     print('Total N/A values this day:'+str(len(nine_thirteen)))
#     print('Percent N/A: '+str(round((len(nine_thirteen))/(len(x))*100,2))+ '%')
#     print(nine_thirteen['buckets'].explode().value_counts()[0:10])

In [ ]:
# date1='2022-01-11'
# date2='2022-01-13'
# na_analysis(date1,date2)

In [ ]:
# print(len(df[(df['attribute'].astype(str)=='fabric_weight_type')&(df['resolution'].astype(str)!='rules')]))
# df[(df['attribute'].astype(str)=='fabric_weight_type')&(df['resolution'].astype(str)!='rules')][0:500]#['value'].explode().value_counts()

In [ ]:
# date1='2021-12-17'
# date2='2021-12-19'
# na_analysis(date1,date2)

In [ ]:
# date1='2021-11-14'
# date2='2021-11-15'
# na_analysis(date1,date2)

In [ ]:
# date1='2021-11-14'
# date2='2021-11-15'
# na_analysis(date1,date2)

In [ ]:
# date1='2021-11-15'
# date2='2021-11-16'
# na_analysis(date1,date2)

In [ ]:
# date1='2021-11-20'
# date2='2021-11-20'
# na_analysis(date1,date2)

In [ ]:
# date1='2021-11-22'
# date2='2021-11-22'
# na_analysis(date1,date2)

In [ ]:
# date1='2021-11-29'
# date2='2021-12-04'
# na_analysis(date1,date2)

In [ ]:
# date1='2021-12-06'
# date2='2021-12-06'
# na_analysis(date1,date2)

In [ ]:
# print(len(bb[(bb['value'].astype(str)=='n/a')&(bb['attribute'].astype(str)=='material')]))
# bb[(bb['value'].astype(str)=='n/a')&(bb['attribute'].astype(str)=='material')]['buckets'].explode().value_counts()

In [ ]:
# bb=df[(df['curated_date'].astype(str)>'2021-11-29')&(df['curated_date'].astype(str)<'2021-12-01')]
# print(bb['curated_date'].explode().value_counts())
# print(len(bb))
# bb['curation_tasks.curated_by'].explode().value_counts()

In [ ]:
# print(len(bb[(bb['curation_tasks.curated_by'].astype(str)!='762.0')]))
# print(len(bb[(bb['curation_tasks.curated_by'].astype(str)!='762.0')&(bb['value'].astype(str)=='n/a')&(bb['buckets'].astype(str)=='Rugs & Mats')]))
# bb[(bb['curation_tasks.curated_by'].astype(str)!='762.0')&(bb['value'].astype(str)=='n/a')&(bb['buckets'].astype(str)=='Rugs & Mats')]['attribute'].explode().value_counts()[0:500]